In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'


# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]
wds = ['市区町村コード','市区町村名','大字町丁目コード','大字町丁目名']
c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[wds]

In [2]:
c_code

,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,13101,千代田区,131010001002,内幸町２丁目
1,13101,千代田区,131010001001,内幸町１丁目
2,13101,千代田区,131010002003,霞が関３丁目
3,13101,千代田区,131010003000,北の丸公園
4,13101,千代田区,131010004001,富士見１丁目
...,...,...,...,...
5359,13421,小笠原村,134210003000,母島
5360,13421,小笠原村,134210004000,父島
5361,13421,小笠原村,134210005000,聟島
5362,13421,小笠原村,134210006000,南鳥島


In [3]:
d2k = {
    '１丁':'一丁',
    '２丁':'二丁',
    '３丁':'三丁',
    '４丁':'四丁',
    '５丁':'五丁',
    '６丁':'六丁',
    '７丁':'七丁',
    '８丁':'八丁',
    '９丁':'九丁'
}

sd2k = {
    '1丁':'１丁',
    '2丁':'２丁',
    '3丁':'３丁',
    '4丁':'４丁',
    '5丁':'５丁',
    '6丁':'６丁',
    '7丁':'７丁',
    '8丁':'８丁',
    '9丁':'９丁'
}

## 1. 大田区

In [4]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata = pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

ohta_finedata['区']='大田区'

for i in d2k.keys():
    ohta_finedata['町丁目名']=ohta_finedata['町丁目名'].str.replace(i,d2k[i])


In [5]:
#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

## 2. 品川

In [6]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['丁目名','年齢','人口']
mer=['丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)
shina_finedata['区']='品川区'

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata = pd.merge(shina_finedata, c_code, left_on=['区','町丁目名'], right_on=['市区町村名','大字町丁目名'])



for i in d2k.keys():
    shina_finedata['町丁目名']=shina_finedata['町丁目名'].str.replace(i,d2k[i])


In [7]:
#shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

## 3. 目黒

In [8]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]
meg_finedata = pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

meg_finedata['区']='目黒区'

for i in d2k.keys():
    meg_finedata['町丁目名']=meg_finedata['町丁目名'].str.replace(i,d2k[i])


In [9]:
#meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

## 4. 港区

In [10]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [11]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [12]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:21,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)

mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata = pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

mina_finedata['区']='港区'

for i in d2k.keys():
    mina_finedata['町丁目名']=mina_finedata['町丁目名'].str.replace(i,d2k[i])



In [13]:
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

## 5. 渋谷

In [14]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月_rev.xlsx')

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

shib_finedata = pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名')

shib_finedata['区']='渋谷区'

for i in d2k.keys():
    shib_finedata['町丁目名']=shib_finedata['町丁目名'].str.replace(i,d2k[i])


In [15]:
shib_finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区
0,恵比寿一丁目,0,25,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
1,恵比寿一丁目,1,36,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
2,恵比寿一丁目,2,31,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
3,恵比寿一丁目,3,37,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
4,恵比寿一丁目,4,17,13113,渋谷区,131130010001,恵比寿１丁目,渋谷区
...,...,...,...,...,...,...,...,...
11654,神宮前六丁目,97,1,13113,渋谷区,131130001006,神宮前６丁目,渋谷区
11655,神宮前六丁目,98,1,13113,渋谷区,131130001006,神宮前６丁目,渋谷区
11656,神宮前六丁目,99,2,13113,渋谷区,131130001006,神宮前６丁目,渋谷区
11657,神宮前六丁目,102,1,13113,渋谷区,131130001006,神宮前６丁目,渋谷区


In [16]:
#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

### 6. 台東

In [86]:
tai_data = pd.DataFrame()
for i in ['１','２','３']:
    tai = pd.read_excel(prefix+'町丁目名年齢年代別人口分布（台東）その'+i+'_確認済み.xlsx')
    tai_data = pd.concat([tai_data, tai])

tai_data['区']='台東区'
tai_data = tai_data.drop(columns='行政コード')

tc = {'level_2':'世代', 0:'人口'}
tmp_tai = tai_data.set_index(['区','町丁目名'])
tt = pd.DataFrame(tmp_tai.stack().reset_index()).rename(columns=tc)
tai_data = pd.merge(tt, c_code, left_on=['区','町丁目名'], right_on=['市区町村名','大字町丁目名'])

#for i in d2k.keys():
#    tai_data['町丁目名']=tai_data['町丁目名'].str.replace(i,d2k[i])
    
tai_finedata = tai_data

In [87]:
tai_finedata.to_excel(prefix + dataname + '（台東）.xlsx', index=False)

In [88]:
tai_finedata

,区,町丁目名,世代,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名
0,台東区,台東１丁目,0~4歳,60,13106,台東区,131060007001,台東１丁目
1,台東区,台東１丁目,5~9歳,31,13106,台東区,131060007001,台東１丁目
2,台東区,台東１丁目,10~14歳,57,13106,台東区,131060007001,台東１丁目
3,台東区,台東１丁目,15~19歳,59,13106,台東区,131060007001,台東１丁目
4,台東区,台東１丁目,20~24歳,113,13106,台東区,131060007001,台東１丁目
...,...,...,...,...,...,...,...,...
2695,台東区,谷中７丁目,100歳~104歳,0,13106,台東区,131060030007,谷中７丁目
2696,台東区,谷中７丁目,105歳~109歳,0,13106,台東区,131060030007,谷中７丁目
2697,台東区,谷中７丁目,110歳~114歳,0,13106,台東区,131060030007,谷中７丁目
2698,台東区,谷中７丁目,115歳~119歳,0,13106,台東区,131060030007,谷中７丁目


### 7. 中央

In [19]:
chu_data = pd.read_excel(prefix + '中央区2022年3月.xlsx', dtype={'丁目':str}).rename(columns={'総数':'人口'})

chu_data = chu_data.fillna('')
chu_data['町丁目名']=''


In [20]:
for i, r in chu_data.iterrows():
    k = r['丁目']
    if k!='':
        chu_data.loc[i,'町丁目名'] = r['地域'] + sd2k[str(k) + '丁'] +'目'
    else:
        chu_data.loc[i,'町丁目名'] = r['地域']

chu_data['世代']='全世代'
chu_data['区'] = '中央区'
chu_data['gord'] = 99

In [21]:
#chu_data = pd.merge(chu_data, c_code, left_on='町丁目名', right_on='大字町丁目名')
chu_finedata = chu_data[['区','町丁目名','世代','人口']]

In [22]:
for i in d2k.keys():
    chu_finedata['町丁目名']=chu_finedata['町丁目名'].str.replace(i,d2k[i])


/var/folders/pt/vj2jdzds3l1_9b69_2m9tt980000gn/T/ipykernel_27791/3685776544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chu_finedata['町丁目名']=chu_finedata['町丁目名'].str.replace(i,d2k[i])


In [23]:
#chu_finedata.to_excel(prefix + dataname + '（中央）.xlsx', index=False)

In [68]:
chu_finedata.head()

,区,町丁目名,世代,人口
0,中央区,八重洲二丁目,全世代,35
1,中央区,京橋一丁目,全世代,66
2,中央区,京橋二丁目,全世代,106
3,中央区,京橋三丁目,全世代,68
4,中央区,銀座一丁目,全世代,1473


In [25]:
#chu_finedata.to_csv(prefix + 'final_chu_data.csv', index=False)
# chu_finedata.to_csv(prefix + dataname + '（渋谷）.xlsx', index=False)

### データ統合

In [67]:
finedata

,町丁目名,年齢,人口,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,区,世代
0,大森東一丁目,0.0,24,13111,大田区,131110039001,大森東１丁目,大田区,
1,大森東一丁目,1.0,28,13111,大田区,131110039001,大森東１丁目,大田区,
2,大森東一丁目,2.0,36,13111,大田区,131110039001,大森東１丁目,大田区,
3,大森東一丁目,3.0,36,13111,大田区,131110039001,大森東１丁目,大田区,
4,大森東一丁目,4.0,31,13111,大田区,131110039001,大森東１丁目,大田区,
...,...,...,...,...,...,...,...,...,...
69684,谷中７丁目,NaN,0,13106,台東区,131060030007,谷中７丁目,台東区,
69685,谷中７丁目,NaN,0,13106,台東区,131060030007,谷中７丁目,台東区,
69686,谷中７丁目,NaN,0,13106,台東区,131060030007,谷中７丁目,台東区,
69687,谷中７丁目,NaN,0,13106,台東区,131060030007,谷中７丁目,台東区,


In [89]:
finedata = pd.concat([ohta_finedata,
                      meg_finedata,
                      shib_finedata,
                      shina_finedata
                      ]).reset_index(drop=True)

finedata['世代'] = ''

#finedata = pd.concat([finedata, chu_finedat#a])

# fiveband = list(mina_finedata.loc[:20,:]['世代'])
#print(fiveband)

# ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
# '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
# '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳'
# '105~109歳', '110~114歳', '115~119歳', '120歳以上']

# 0~130まで5づつでビンを作成
bin_band =  list(range(0,131,5))
# binのラベル
fiveband = ['0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳',
 '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳',
 '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100~104歳',
 '105~109歳', '110~114歳', '115~119歳', '120~124歳', '125~129歳']#, '130歳以上']

fiveband_df = pd.DataFrame(
            {
                'gord' : list(range(0,26)),
                '世代': fiveband
            }
)


finedatas = finedata[finedata['人口']>0]

finedata['世代']=pd.cut(finedata['年齢'], bin_band, right=False, labels=fiveband)

finedata = pd.merge(finedata, fiveband_df, on='世代', how='left')


In [90]:
#fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
fd_grp = ['区','町丁目名','大字町丁目コード','世代','gord']
srt = ['大字町丁目コード','gord']

final = pd.DataFrame(finedata.groupby(fd_grp)['人口'].agg(sum)).sort_values(srt).reset_index()

mina_final = pd.merge(mina_finedata, fiveband_df, on='世代')[fd_grp+['人口']]
tai_final = pd.merge(tai_finedata, fiveband_df, on='世代')[fd_grp+['人口']]

final = pd.concat([final, mina_final, tai_final])

In [91]:
final

,区,町丁目名,大字町丁目コード,世代,gord,人口
0,目黒区,三田一丁目,131030019001,0~4歳,0,76
1,目黒区,三田一丁目,131030019001,5~9歳,1,60
2,目黒区,三田一丁目,131030019001,10~14歳,2,47
3,目黒区,三田一丁目,131030019001,15~19歳,3,57
4,目黒区,三田一丁目,131030019001,20~24歳,4,90
...,...,...,...,...,...,...
2155,台東区,谷中３丁目,131060030003,95~99歳,19,9
2156,台東区,谷中４丁目,131060030004,95~99歳,19,1
2157,台東区,谷中５丁目,131060030005,95~99歳,19,5
2158,台東区,谷中６丁目,131060030006,95~99歳,19,2


In [92]:
#　エラーが出る町丁目名
final['町丁目名'] = final['町丁目名'].str.replace('松濤','松涛')
final['町丁目名'] = final['町丁目名'].str.replace('鶯谷町','鴬谷町')

In [93]:
final = final.astype({'大字町丁目コード':str})

In [94]:
d2k = {
    '１丁':'一丁',
    '２丁':'二丁',
    '３丁':'三丁',
    '４丁':'四丁',
    '５丁':'五丁',
    '６丁':'六丁',
    '７丁':'七丁',
    '８丁':'八丁',
    '９丁':'九丁'
}

for i in d2k.keys():
    final['町丁目名']=final['町丁目名'].str.replace(i,d2k[i])


In [95]:
final.to_csv(prefix+'final_data.csv', encoding='utf-8', index=False)
#chu_finedata.to_csv(prefix+'final_chu_data.csv', encoding='utf-8', index=False)
final.to_excel(prefix+"final.xlsx",index=False)

### ヒートマップ

In [ ]:
import streamlit_folium as st_f

In [ ]:
tmp_kgdf[tmp_kgdf['geometry'].isnull()]

In [ ]:
tmp_kgdf[tmp_kgdf['S_NAME'].str.contains('令和')]

In [ ]:
pop = final[final['区']==w]
qoq = pd.DataFrame()
for g in fb:
    qoq = pd.concat([qoq, pop[pop['世代']==g]])
        
qoq.groupby(['町丁目名'])['人口'].sum().reset_index()

In [ ]:
result

In [85]:
import geopandas as gpd
import folium
import matplotlib
import mapclassify
import numpy as np

geodir = d + '/../geoshape/'

ward=['港区','目黒区','大田区','品川区','渋谷区','台東区']
w=ward[5]
fb=fiveband[5:8]
gen = ','.join(fb)
print(w, gen)


pop = final[final['区']==w]
qoq = pd.DataFrame()
for g in fb:
    qoq = pd.concat([qoq, pop[pop['世代']==g]])
        
qoq = qoq.groupby(['区','町丁目名'])['人口'].sum().reset_index()
        
gdf = gpd.read_file(geodir + w + '.geojson')
gdf = gdf[gdf['HCODE']==8101]
result = pd.merge(qoq, gdf, left_on=['区','町丁目名'], right_on=['CITY_NAME','S_NAME']) # , how='left')
result['世代'] = gen

#kgdf = kgdf[kgdf['区']==w]

result = gpd.GeoDataFrame(result)


#tmp_kgdf.explore(column=tmp_kgdf['人口'],cmap='Reds',tooltip=['町丁目名','人口'],tiles='CartoDB positron')
result.explore(column=result['人口'],cmap='Reds',tooltip=['町丁目名','世代','人口'],tiles='CartoDB positron')
#tmp_kgdf.info()


台東区 25~29歳,30~34歳,35~39歳


In [ ]:
pop

In [ ]:
gdf[['KEYCODE2','KEYCODE1','KIHON1','KIHON2','KCODE1','PREF','CITY','S_AREA','S_NAME']].head()

In [ ]:
#c_code[c_code['大字町丁目コード']=='131030001001']
#c_code.head()

gdf['S_NAME']=gdf['S_NAME'].str.replace('一丁','１丁').str.replace('二丁','２丁')

In [ ]:
gdf['おお']=''

In [ ]:
gdf['S_NAME']

In [ ]:
#gdf[(gdf['CITY_NAME']=='港区') and (gdf['S_NAME']=='芝一丁目')]
gdf[(gdf['S_NAME']=='芝一丁目')]

In [ ]:
final[(final['町丁目名']=='芝１丁目')]

In [122]:
rf = pd.read_csv(prefix + 'Ward_data_reference_date.csv')

dict(zip(list(rf['区名']),list(rf['データ基準日'])))